In [1]:
# 
import cv2

In [2]:
print(cv2.__version__)

4.3.0


In [3]:
import numpy as np

In [4]:
img = np.random.randint(0, 10, (7, 7))

print(img)

[[2 1 5 4 8 8 1]
 [2 1 5 1 7 0 5]
 [4 2 3 0 8 3 0]
 [2 6 0 1 5 4 1]
 [6 0 8 9 6 3 0]
 [0 3 1 1 7 1 2]
 [6 0 3 1 2 5 1]]


In [5]:
img_np = np.ones([7,7])

print(img_np)

[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [6]:
kernel = np.array([-1, 0, 1])

print(kernel)

[-1  0  1]


In [7]:
print(type(kernel))
print(type(img_np))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [8]:
img_cv = cv2.resize(img_np, (7,7))

print(img_cv)

[[1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]]


In [9]:
kernel_cv = cv2.resize(kernel, (1, 3))

print(kernel)

[-1  0  1]


In [10]:
print(type(img_cv))
print(type(kernel_cv))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [11]:
conv_test = cv2.filter2D(img_cv, -1, kernel)

print(conv_test)

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]


In [12]:
img_cv[0][0] = 0
img_cv[3][3] = 2
img_cv[6][6] = 3

# filter2D - Convolution이 결국 라플라스 변환과 
#            푸리에 변환에 관계를 가지고 있기 때문
conv_test = cv2.filter2D(img_cv, -1, kernel)

print(conv_test)

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [13]:
# 라플라스 변환 => 전달 함수를 얻기 위해 계산함
# 입력 대 출력비를 구하겠다.
# 입력이 10, 출력이 7, 입출력비: 7/10
# 입출력비가 미분 방정식 형태이면, 위치마다 기울기가 달라서 비율이 구해질 수 없음
# 입력이 y'' = y'+ 3y + 2, 출력이 y' = 3e^3x => e^3x
# 라플라스 변환을 통해서 입출력비를 계산할 수 있게 된다.

# Low Pass Filter(LPF 설계법) - 1D Convolution
# 전달함수 -> 실제 값을 필터링 하는 방법

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [15]:
classifier = Sequential()
classifier.add(
    Conv2D(
        # 3x3인 특성 탐지기 32개
        32, (3,3), 
        # 64x64는 변환된 이미지 크기, 노트북 CPU에서는 64x64 권장
        input_shape=(64, 64, 3), 
        activation='relu'
    )
)
classifier.add(
    Conv2D(
        64, (3, 3), 
        activation = 'relu'
    )
)

classifier.add(
    Conv2D(
        128, (3, 3), 
        activation = 'relu'
    )
)

# 2x2행렬 블록에서 가장 큰 수 선택
classifier.add(MaxPool2D(pool_size=(2,2)))
# classifier.add(MaxPool2D(pool_size = (2, 2)))

# CNN에서 ANN 입력 형태로 변환하는 영역
# 풀링된 이미지를 펴서 하나의 열로 생성
classifier.add(Flatten())

In [16]:
classifier.add(
    Dense(256, activation = 'relu')
)
classifier.add(
    Dense(128, activation = 'relu')
)
classifier.add(
    Dense(64, activation = 'relu')
)
classifier.add(
    Dense(32, activation = 'relu')
)
classifier.add(
    Dense(1, activation = 'sigmoid')
)

classifier.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [17]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_gen = ImageDataGenerator(rescale = 1.0 / 255)

In [18]:
trainSet = train_gen.flow_from_directory(
    'Applied-Deep-Learning-with-Keras/Lesson07/' +
    'Datasets/dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

testSet = test_gen.flow_from_directory(
    'Applied-Deep-Learning-with-Keras/Lesson07/' +
    'Datasets/dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [23]:
batch_size = 100

classifier.fit_generator(
    trainSet,
    steps_per_epoch = int(10000/batch_size),
    epochs = 15,
    validation_data = testSet,
    validation_steps = int(2500/batch_size)

)

Epoch 1/15
100/100 [==============================] - 31s 309ms/step - loss: 0.6763 - accuracy: 0.5891 - val_loss: 0.6788 - val_accuracy: 0.5775
Epoch 2/15
100/100 [==============================] - 31s 308ms/step - loss: 0.6496 - accuracy: 0.6272 - val_loss: 0.6060 - val_accuracy: 0.6762
Epoch 3/15
100/100 [==============================] - 31s 309ms/step - loss: 0.6248 - accuracy: 0.6569 - val_loss: 0.6129 - val_accuracy: 0.6675
Epoch 4/15
100/100 [==============================] - 31s 310ms/step - loss: 0.6015 - accuracy: 0.6841 - val_loss: 0.5863 - val_accuracy: 0.6963
Epoch 5/15
100/100 [==============================] - 31s 314ms/step - loss: 0.5809 - accuracy: 0.6853 - val_loss: 0.5414 - val_accuracy: 0.7300
Epoch 6/15
100/100 [==============================] - 31s 308ms/step - loss: 0.5665 - accuracy: 0.7103 - val_loss: 0.5332 - val_accuracy: 0.7312
Epoch 7/15
100/100 [==============================] - 31s 310ms/step - loss: 0.5540 - accuracy: 0.7091 - val_loss: 0.5381 - val_ac

In [24]:
import numpy as np

from keras.preprocessing import image

In [25]:
new_image = image.load_img(
    'test_image_2.jpg',
    target_size = (64, 64)
)

In [26]:
new_image = image.img_to_array(new_image)
new_image = np.expand_dims(new_image, axis = 0)

result = classifier.predict(new_image)
#trainSet.class_indices

if result[0][0] == 1:
    prediction = 'Dog'
else:
    prediction = 'Cat'
    
print(prediction)

Cat
